In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 

In [2]:
df = pd.read_csv('E:\diamonds.csv', index_col = 0)

In [3]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


Найдем нули и уберем их

In [5]:
df0 = df[~df.eq(0).any(1)]
df0.shape

(53920, 10)

Выделим отдельно категориальные фичи и те, к которым мы хотим сделать полиномиальный фичинг. Также отдельно остивим фичи, где будем применять StandartScaler. Также выделим целевую переменную

In [6]:
y = df0.price
X = df0.drop('price', axis = 1)
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75


Разделим на train/test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

In [9]:
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z
7653,0.90,Ideal,G,VS2,62.6,57.0,6.14,6.20,3.86
4220,0.90,Very Good,F,SI2,62.4,58.0,6.15,6.19,3.85
14104,1.21,Premium,F,SI1,61.1,61.0,6.78,6.74,4.13
1186,0.70,Very Good,G,VVS2,60.8,57.0,5.72,5.76,3.49
49716,0.76,Premium,J,VS1,61.1,58.0,5.85,5.91,3.59


Следующие три блока делают одно и тоже, просто признаки ставновятся в разный порядок, если их позапускать по отдельности, можно получить совершено разные результаты

In [10]:
poly_features = ['x','y','z']
num_features = ['carat','depth','table'] #добавить x,y,z
cat_features = ['cut','color','clarity']
# И где здесь логическая ошибка?
tsfc = ColumnTransformer([
        ("pol", PolynomialFeatures(2,include_bias=False), poly_features),
        ("num", StandardScaler(), num_features),   #поменять строки с pol и num местами
        ("cat", OneHotEncoder(), cat_features),
        ])

tsfc.fit(X_train)
X_train_tf = tsfc.transform (X_train)
X_test_tf  = tsfc.transform (X_test)
X_train_tf

array([[6.14, 6.2 , 3.86, ..., 1.  , 0.  , 0.  ],
       [6.15, 6.19, 3.85, ..., 0.  , 0.  , 0.  ],
       [6.78, 6.74, 4.13, ..., 0.  , 0.  , 0.  ],
       ...,
       [4.73, 4.76, 2.94, ..., 0.  , 0.  , 1.  ],
       [6.13, 6.03, 3.82, ..., 0.  , 0.  , 0.  ],
       [6.78, 6.79, 4.27, ..., 0.  , 0.  , 0.  ]])

In [11]:
X_test_tf

array([[4.36, 4.39, 2.66, ..., 0.  , 0.  , 0.  ],
       [6.97, 7.02, 4.32, ..., 0.  , 0.  , 0.  ],
       [7.36, 7.27, 4.58, ..., 0.  , 0.  , 0.  ],
       ...,
       [7.06, 7.02, 4.33, ..., 0.  , 0.  , 0.  ],
       [8.64, 8.57, 5.34, ..., 0.  , 0.  , 0.  ],
       [4.58, 4.61, 2.76, ..., 0.  , 0.  , 0.  ]])

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit



cat_features = ['cut','color','clarity']
poly_features = ['x','y','z']
num_features = ['carat','depth','table','x','y','z']

tsf = ColumnTransformer([
        ("num", StandardScaler(), num_features),
        ("pol", PolynomialFeatures(2,include_bias=False), poly_features),
        ("cat", OneHotEncoder(), cat_features)
        ])

pipeline = Pipeline([
        ('ts', tsf),
        ('reg', ElasticNet(tol = 0.001, max_iter = 10000))
    ])


param_grid = [{
     'reg__l1_ratio': [0.5, 0.9, 0.99, 1],
     'reg__alpha': [10**(-3), 10**(-1), 1]
            }]
cv = ShuffleSplit(n_splits=5, test_size=.25, random_state=0)

grid = GridSearchCV(pipeline, param_grid, cv=cv, return_train_score=True, n_jobs=-1)
grid.fit(X_train,y_train)

gridresults = pd.DataFrame(grid.cv_results_)
display(gridresults.sort_values(["rank_test_score"]).T)

,8,9,4,10,5,6,0,11,7,1,2,3
mean_fit_time,5.29085,18.754,49.3982,43.0851,51.2237,50.5207,46.8746,40.4796,46.9881,45.7376,45.7797,46.0565
std_fit_time,1.1615,8.14539,0.784203,7.39777,1.14928,1.84271,0.643697,5.24274,0.535211,0.151172,0.097162,0.365179
mean_score_time,0.0363007,0.0403475,0.0363862,0.0436057,0.0339101,0.033309,0.0330925,0.0281111,0.0325785,0.028789,0.02839,0.0370982
std_score_time,0.00692653,0.0125154,0.0066456,0.0114834,0.00398953,0.00223872,0.00688796,0.0116875,0.00228438,0.00143272,0.00147777,0.0123654
param_reg__alpha,1,1,0.1,1,0.1,0.1,0.001,1,0.1,0.001,0.001,0.001
param_reg__l1_ratio,0.5,0.9,0.5,0.99,0.9,0.99,0.5,1,1,0.9,0.99,1
params,"{'reg__alpha': 1, 'reg__l1_ratio': 0.5}","{'reg__alpha': 1, 'reg__l1_ratio': 0.9}","{'reg__alpha': 0.1, 'reg__l1_ratio': 0.5}","{'reg__alpha': 1, 'reg__l1_ratio': 0.99}","{'reg__alpha': 0.1, 'reg__l1_ratio': 0.9}","{'reg__alpha': 0.1, 'reg__l1_ratio': 0.99}","{'reg__alpha': 0.001, 'reg__l1_ratio': 0.5}","{'reg__alpha': 1, 'reg__l1_ratio': 1}","{'reg__alpha': 0.1, 'reg__l1_ratio': 1}","{'reg__alpha': 0.001, 'reg__l1_ratio': 0.9}","{'reg__alpha': 0.001, 'reg__l1_ratio': 0.99}","{'reg__alpha': 0.001, 'reg__l1_ratio': 1}"
split0_test_score,0.855248,0.889191,0.904318,0.922029,0.922439,0.926051,0.926026,0.925862,0.92537,0.9255,0.925237,0.925199
split1_test_score,0.853512,0.888776,0.904216,0.921851,0.922246,0.925429,0.92539,0.925286,0.924992,0.925057,0.9249,0.924875
split2_test_score,-1.16787,-4.12309,-5.76391,-7.11042,-7.91762,-11.1865,-15.1237,-15.4482,-21.442,-21.817,-23.5195,-23.6614


In [13]:
grid.best_params_

{'reg__alpha': 1, 'reg__l1_ratio': 0.5}